In [16]:
!pip install pycocotools

  Using cached pycocotools-2.0.6-cp37-cp37m-linux_x86_64.whl


In [56]:
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 23.09 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [ ]:
   # Interface for evaluating detection on the Microsoft COCO dataset.
    #
    # The usage for CocoEval is as follows:
    #  cocoGt=..., cocoDt=...       # load dataset and results
    #  E = CocoEval(cocoGt,cocoDt); # initialize CocoEval object
    #  E.params.recThrs = ...;      # set parameters as desired
    #  E.evaluate();                # run per image evaluation
    #  E.accumulate();              # accumulate per image results
    #  E.summarize();               # display summary metrics of results
    # For example usage see evalDemo.m and http://mscoco.org/.
    #
    # The evaluation parameters are as follows (defaults in brackets):
    #  imgIds     - [all] N img ids to use for evaluation
    #  catIds     - [all] K cat ids to use for evaluation
    #  iouThrs    - [.5:.05:.95] T=10 IoU thresholds for evaluation
    #  recThrs    - [0:.01:1] R=101 recall thresholds for evaluation
    #  areaRng    - [...] A=4 object area ranges for evaluation
    #  maxDets    - [1 10 100] M=3 thresholds on max detections per image
    #  iouType    - ['segm'] set iouType to 'segm', 'bbox' or 'keypoints'
    #  iouType replaced the now DEPRECATED useSegm parameter.
    #  useCats    - [1] if true use category labels for evaluation
    # Note: if useCats=0 category labels are ignored as in proposal scoring.
    # Note: multiple areaRngs [Ax2] and maxDets [Mx1] can be specified.
    #
    # evaluate(): evaluates detections on every image and every category and
    # concats the results into the "evalImgs" with fields:
    #  dtIds      - [1xD] id for each of the D detections (dt)
    #  gtIds      - [1xG] id for each of the G ground truths (gt)
    #  dtMatches  - [TxD] matching gt id at each IoU or 0
    #  gtMatches  - [TxG] matching dt id at each IoU or 0
    #  dtScores   - [1xD] confidence of each dt
    #  gtIgnore   - [1xG] ignore flag for each gt
    #  dtIgnore   - [TxD] ignore flag for each dt at each IoU
    #
    # accumulate(): accumulates the per-image, per-category evaluation
    # results in "evalImgs" into the dictionary "eval" with fields:
    #  params     - parameters used for evaluation
    #  date       - date evaluation was performed
    #  counts     - [T,R,K,A,M] parameter dimensions (see above)
    #  precision  - [TxRxKxAxM] precision for every evaluation setting
    #  recall     - [TxKxAxM] max recall for every evaluation setting
    # Note: precision and recall==-1 for settings with no gt objects.
    #
    # See also coco, mask, pycocoDemo, pycocoEvalDemo
    #
    # Microsoft COCO Toolbox.      version 2.0
    # Data, paper, and tutorials available at:  http://mscoco.org/
    # Code written by Piotr Dollar and Tsung-Yi Lin, 2015.
    # Licensed under the Simplified BSD License [see coco/license.txt]

In [17]:
!pwd

/home/jupyter/visualization


In [8]:
%cd ..

/home/jupyter/visualization


In [1]:
import pycocotools.cocoeval
import pycocotools.coco
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
pred_path = '/home/jupyter/data/prediction.json'
gt_path = '/home/jupyter/data/ground_truth.json'
img_path = '/home/jupyter/data/images'

In [36]:
# coco_gt, coco_pred 선언법
# 주석에는 파일명으로 하라고 하는데 코드보면 numpy array로 넣어줘도 사용 가능
coco_gt = COCO(gt_path)
coco_pred = coco_gt.loadRes(pred_path)

coco_eval = COCOeval(coco_gt, coco_pred, 'bbox')

catIds = range(34)
# 각종 파라미터 설정
coco_eval.params.catIds = catIds
# params.catIds 설정하면 class 별 mAP 출력해주는듯

# 수치 뽑는 3종셋트
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# coco_eval.stats에 접근하면 summarize 시에 할당된 수치를 받아올 수 있다.

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.54s).
Accumulating evaluation results...
DONE (t=0.21s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.908
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.908
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.908
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.908
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.902
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.910
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDe

In [13]:
from tabulate import tabulate
import itertools
import numpy as np

def _derive_coco_results(coco_eval, iou_type, class_names=None):
        """
        Derive the desired score numbers from summarized COCOeval.

        Args:
            coco_eval (None or COCOEval): None represents no predictions from model.
            iou_type (str):
            class_names (None or list[str]): if provided, will use it to predict
                per-category AP.

        Returns:
            a dict of {metric name: score}
        """

        metrics = {
            "bbox": ["AP", "AP50", "AP75", "APs", "APm", "APl"],
            "segm": ["AP", "AP50", "AP75", "APs", "APm", "APl"],
            "keypoints": ["AP", "AP50", "AP75", "APm", "APl"],
        }[iou_type]

        if coco_eval is None:
            print("No predictions from the model!")

        # the standard metrics
        results = {
            metric: float(coco_eval.stats[idx] * 100 if coco_eval.stats[idx] >= 0 else "nan")
            for idx, metric in enumerate(metrics)
        }
        print(
            "Evaluation results for {}: \n".format(iou_type) + create_small_table(results)
        )
        if not np.isfinite(sum(results.values())):
            print("Some metrics cannot be computed and is shown as NaN.")

        # Compute per-category AP
        # from https://github.com/facebookresearch/Detectron/blob/a6a835f5b8208c45d0dce217ce9bbda915f44df7/detectron/datasets/json_dataset_evaluator.py#L222-L252 # noqa
        precisions = coco_eval.eval["precision"]
        # precision has dims (iou, recall, cls, area range, max dets)
        assert len(class_names) == precisions.shape[2]

        results_per_category = []
        for idx, name in enumerate(class_names):
            # area range index 0: all area ranges
            # max dets index -1: typically 100 per image
            precision = precisions[:, :, idx, 0, -1]
            precision = precision[precision > -1]
            ap = np.mean(precision) if precision.size else float("nan")
            results_per_category.append(("{}".format(name), float(ap * 100)))

        # tabulate it
        N_COLS = min(6, len(results_per_category) * 2)
        results_flatten = list(itertools.chain(*results_per_category))
        results_2d = itertools.zip_longest(*[results_flatten[i::N_COLS] for i in range(N_COLS)])
        table = tabulate(
            results_2d,
            tablefmt="pipe",
            floatfmt=".3f",
            headers=["category", "AP"] * (N_COLS // 2),
            numalign="left",
        )
        print("Per-category {} AP: \n".format(iou_type) + table)
        
def create_small_table(small_dict):
    """
    Create a small table using the keys of small_dict as headers. This is only
    suitable for small dictionaries.

    Args:
        small_dict (dict): a result dictionary of only a few items.

    Returns:
        str: the table as a string.
    """
    keys, values = tuple(zip(*small_dict.items()))
    table = tabulate(
        [values],
        headers=keys,
        tablefmt="pipe",
        floatfmt=".3f",
        stralign="center",
        numalign="center",
    )
    return table

In [18]:
classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019",
           "chevrolet_trailblazer_suv_2021_", "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", "genesis_g80_sedan_2021_", 
           "genesis_gv80_suv_2020_", "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", "hyundai_grandeur_sedan_2011_2016", 
           "hyundai_grandstarex_van_2018_2020", "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", 
           "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", 
           "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", "kia_morning_hatchback_2004_2010", 
           "kia_morning_hatchback_2011_2016", "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", "kia_sorrento_suv_2020_", 
           "kia_soul_suv_2014_2018", "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_",
           "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]

_derive_coco_results(coco_eval = coco_eval, iou_type = "bbox", class_names=classes)

Evaluation results for bbox: 
|   AP   |  AP50  |  AP75  |  APs  |  APm  |  APl   |
|:------:|:------:|:------:|:-----:|:-----:|:------:|
| 90.806 | 90.807 | 90.805 |  nan  |  nan  | 90.806 |
Some metrics cannot be computed and is shown as NaN.
Per-category bbox AP: 
| category                         | AP      | category                          | AP      | category                          | AP      |
|:---------------------------------|:--------|:----------------------------------|:--------|:----------------------------------|:--------|
| chevrolet_malibu_sedan_2012_2016 | 0.264   | chevrolet_malibu_sedan_2017_2019  | 0.000   | chevrolet_trailblazer_suv_2021_   | 100.000 |
| chevrolet_trax_suv_2017_2019     | 96.319  | genesis_g80_sedan_2016_2020       | 100.000 | genesis_g80_sedan_2021_           | 100.000 |
| genesis_gv80_suv_2020_           | 100.000 | hyundai_avante_sedan_2011_2015    | 100.000 | hyundai_avante_sedan_2020_        | 100.000 |
| hyundai_grandeur_sedan_2011_2016 | 

In [15]:
import json

#print(type(pred_path))
#print(type(open(pred_path)))
anns = json.load(open(pred_path))
#json_string = json.dumps(anns)
#print(type(anns))
#print(type(json_string))

#print(anns)
for ann in anns['annotations']:
    print(ann['image_id'])

0
1
2
2
3
4
5
5
6
6
7
7
8
9
9
10
10
11
12
12
12
13
14
15
16
16
17
17
17
18
18
19
19
20
21
22
23
23
24
25
26
27
27
27
28
28
29
29
29
30
30
31
31
31
31
32
33
34
34
34
34
34
35
35
36
36
37
37
37
37
38
38
38
39
39
40
41
41
42
42
43
44
44
45
45
46
46
47
48
48
49
49
50
51
51
52
52
53
53
54
54
55
55
55
55
56
57
58
58
58
58
58
58
59
59
59
60
60
61
61
61
61
61
62
62
63
63
64
65
65
66
67
68
69
70
70
70
71
72
73
73
74
75
75
75
76
77
77
78
78
78
79
79
80
81
81
81
81
82
82
82
82
83
83
84
85
85
86
87
87
88
88
89
90
90
90
90
90
90
91
91
92
93
94
95
96
97
98
98
98
99
100
101
101
101
101
102
103
104
104
104
105
106
106
106
107
107
107
108
109
109
110
110
111
111
111
112
112
112
113
114
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
122
122
122
123
123
124
124
125
125
126
127
127
127
128
128
128
129
129
130
130
131
132
133
133
134
134
135
136
136
137
137
138
138
138
139
139
139
140
140
141
142
142
143
143
144
144
145
146
146
147
147
147
148
149
149
149
149
149
149
149
149
149
150
151
151
15